In [6]:
import os
import csv
import ast
import json
import statistics

import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

from datetime import datetime, timedelta

In [840]:
date = "20240313"
date_with_dash = "2024-03-13"
port = "5201"
phone_time = "0344"
time = "1143"
device = "sm00"
if int(port) % 2 == 0:
    sent_file_name = f"log_{date}_{phone_time}_{port}_client"
    received_file_name = f"log_{date}_{time}_{port}_server"
else:
    sent_file_name = f"log_{date}_{time}_{port}_server"
    received_file_name = f"log_{date}_{phone_time}_{port}_client"
path = f"/Volumes/mollyT7/MOXA/{date_with_dash}/QUIC-10M/{device}/#01/"

In [841]:
sync_file_name = f"/Volumes/mollyT7/MOXA/{date_with_dash}/time_sync_{device}.json"
# sync_file = path + "raw/" + sync_file_name
with open(sync_file_name, 'r') as file:
    data = json.load(file)

# Extract values from the dictionary
values = list(data.values())
mean_diff = values[4] * 1000

### Transform to JSON & CSV file
Process the qlog file to json file & csv file.

In [842]:
def QlogToJsonEntry(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Add commas between lines
    json_str = ",".join(lines)
    # Surround the entire string with square brackets to create a JSON array
    json_str = "[" + json_str + "]"
    # Load the JSON array
    json_entry = json.loads(json_str)
    
    return json_entry

def QlogToJson(json_entry, json_file_path):
    with open(json_file_path, 'w') as json_file:
        json.dump(json_entry, json_file, indent=2)

def JsonToCsv(json_entry, csv_file_path):
     # Open CSV file for writing
    with open(csv_file_path, 'w', newline='') as csv_file:
        # Create a CSV writer
        csv_writer = csv.writer(csv_file)

        # Write header row based on the keys of the second JSON object (assuming at least two objects are present)
        if len(json_entry) >= 2:
            header = list(json_entry[1].keys())
            csv_writer.writerow(header)

            # Write data rows starting from the second object
            for entry in json_entry[1:]:
                csv_writer.writerow(entry.values())

In [843]:
# sender_side_file
sent_raw_path = path + "raw/" + sent_file_name
sent_qlog_file_path = sent_raw_path + ".qlog"
sent_json_file_path = sent_raw_path + ".json"
sent_csv_file_path = sent_raw_path + ".csv"
sent_json_entry = QlogToJsonEntry(sent_qlog_file_path)
# QlogToJson(sent_json_entry, sent_json_file_path)
# JsonToCsv(sent_json_entry, sent_csv_file_path)

In [844]:
received_raw_path = path + "raw/" + received_file_name
received_qlog_file_path = received_raw_path + ".qlog"
received_json_file_path = received_raw_path + ".json"
received_csv_file_path = received_raw_path + ".csv"
received_json_entry = QlogToJsonEntry(received_qlog_file_path)
# QlogToJson(received_json_entry, received_json_file_path)
# JsonToCsv(received_json_entry, received_csv_file_path)

In [845]:
sent_df = pd.read_csv(sent_csv_file_path)
received_df = pd.read_csv(received_csv_file_path)

Set time to UMT+8.

In [846]:
def GetStartTime(json_data):
    # unit: ms
    refTime = json_data[0]["trace"]["common_fields"]["reference_time"]
    return refTime

def ProcessTime(df, reference_time):
    # Extract the "time" values from the DataFrame
    original_times = (df['time'].astype(float))

    # Calculate "epoch_time" and convert to timestamps
    epoch_times = (reference_time + original_times)
    timestamps = pd.to_datetime(epoch_times, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')

    df['epoch_time'] = epoch_times
    df['timestamp'] = timestamps

    return df

In [847]:
# No matter downlink or uplink, the file time that need to change is client side.
if int(port)%2 == 0: # UL
    clientStartTime = GetStartTime(sent_json_entry)
    print(clientStartTime)
    serverStartTime = GetStartTime(received_json_entry)
    print(serverStartTime)

    senderRefTime = clientStartTime + mean_diff
    rcverRefTime = serverStartTime

else:   # DL
    clientStartTime = GetStartTime(received_json_entry)
    print(clientStartTime)
    serverStartTime = GetStartTime(sent_json_entry)
    print(serverStartTime)
    startTimeDiff = (clientStartTime - serverStartTime) + mean_diff

    senderRefTime = serverStartTime
    rcverRefTime = clientStartTime + mean_diff


1710300665342.6233
1710300624377.6797


In [848]:
sent_df = ProcessTime(sent_df, senderRefTime)
# Add 8 hours to both epoch times and timestamps to match UMT+8
# Also sync time with server
epoch_times_gmt8 = sent_df["epoch_time"] + 8 * 3600 * 1000
sent_df["epoch_time"] = epoch_times_gmt8
timestamps_gmt8 = pd.to_datetime(epoch_times_gmt8, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')
sent_df["timestamp"] = timestamps_gmt8

sent_df[-5:]

,time,name,data,epoch_time,timestamp
1035192,450053.778500,recovery:metrics_updated,"{'smoothed_rtt': 20.764, 'latest_rtt': 25.1947...",1.710330e+12,2024-03-13 11:37:54.431458
1035193,450053.782830,recovery:loss_timer_updated,{'event_type': 'cancelled'},1.710330e+12,2024-03-13 11:37:54.431462
1035194,450053.787161,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.431466
1035195,450053.797497,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.431477
1035196,450053.833745,transport:connection_closed,"{'owner': 'remote', 'application_code': 0, 're...",1.710330e+12,2024-03-13 11:37:54.431513


In [849]:
received_df = ProcessTime(received_df, rcverRefTime)
# if the sender is server, then it is no need to calculate time difference
epoch_times_gmt8 = received_df["epoch_time"] + 8 * 3600 * 1000
received_df["epoch_time"] = epoch_times_gmt8
timestamps_gmt8 = pd.to_datetime(epoch_times_gmt8, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')
received_df["timestamp"] = timestamps_gmt8

received_df[-5:]

,time,name,data,epoch_time,timestamp
369676,450051.242745,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.407160
369677,450051.266287,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.407184
369678,450051.321599,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '04...",1.710330e+12,2024-03-13 11:37:54.407239
369679,450051.565609,transport:connection_closed,"{'owner': 'local', 'application_code': 0, 'rea...",1.710330e+12,2024-03-13 11:37:54.407483
369680,450051.612224,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '04...",1.710330e+12,2024-03-13 11:37:54.407530


Parse the data.

In [850]:
# sender side data
metrics_all_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("'bytes_in_flight':"))]
metrics_sent_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("{'bytes_in_flight':"))]
metrics_ack_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("'latest_rtt':"))]
total_sent_rows = sent_df[(sent_df['name'] == 'transport:packet_sent')]
pk_sent_rows = sent_df[(sent_df['name'] == 'transport:packet_sent') & (sent_df['data'].str.contains("'frame_type': 'stream'"))]
rcv_ack_rows = sent_df[(sent_df['name'] == 'transport:packet_received') & (sent_df['data'].str.contains("'frame_type': 'ack'")) & (sent_df['data'].str.contains("'packet_type': '1RTT'"))]
lost_rows = sent_df[sent_df['name'] == 'recovery:packet_lost']

# Get the count of rows
metrics_all_cnt = len(metrics_all_rows)
metrics_c_cnt = len(metrics_sent_rows)
metrics_ack_cnt = len(metrics_ack_rows)
total_sent_cnt = len(total_sent_rows)
pk_sent_cnt = len(pk_sent_rows)
rcv_ack_cnt = len(rcv_ack_rows)
lost_cnt = len(lost_rows)

print("packet_sent: ", pk_sent_cnt, metrics_c_cnt)
print("ack: ", rcv_ack_cnt, metrics_ack_cnt)
print(metrics_all_cnt, metrics_c_cnt, metrics_ack_cnt, pk_sent_cnt, rcv_ack_cnt, lost_cnt)

packet_sent:  281618 281607
ack:  65337 65207
346815 281607 65207 281618 65337 317


In [851]:
pk_rcv_rows = received_df[(received_df['name'] == "transport:packet_received") & (received_df['data'].str.contains("'frame_type': 'stream'"))]
pk_rcv_rows = pk_rcv_rows.reset_index(drop=True)
print(len(pk_rcv_rows))
pk_rcv_rows[:5]

281295


,time,name,data,epoch_time,timestamp
0,56.321354,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412239
1,56.355989,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412273
2,56.374896,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412292
3,60.963594,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.416881
4,60.996302,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.416914


## Deal with sender side data
Concat `transport:packet_sent` & `recovery:metrics_updated`.

In [852]:
metrics_sent_csv_file_path = path + "middle/" + f"sent_metrics_{time}_{port}.csv"
metrics_sent_rows.to_csv(metrics_sent_csv_file_path, index=False)
pk_sent_csv_file_path = path + "middle/" + f"pk_sent_{time}_{port}.csv"
pk_sent_rows.to_csv(pk_sent_csv_file_path, index=False)

In [853]:
def insert(df, idx, new_row):
    df1 = df.iloc[:idx, :]
    df2 = df.iloc[idx:, :]
    df_new = pd.concat([df1, new_row, df2], ignore_index=True)
    return df_new

In [854]:
metrics_sent_rows = metrics_sent_rows.reset_index(drop=True)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)
print(metrics_sent_rows[:5])
print(pk_sent_rows[:5])


        time                      name  \
0   2.081996  recovery:metrics_updated   
1   2.085837  recovery:metrics_updated   
2  27.892317  recovery:metrics_updated   
3  28.165609  recovery:metrics_updated   
4  29.759548  recovery:metrics_updated   

                                                data    epoch_time  \
0  {'bytes_in_flight': 1161, 'packets_in_flight': 2}  1.710329e+12   
1  {'bytes_in_flight': 1252, 'packets_in_flight': 3}  1.710329e+12   
2   {'bytes_in_flight': 310, 'packets_in_flight': 1}  1.710329e+12   
3  {'bytes_in_flight': 1331, 'packets_in_flight': 2}  1.710329e+12   
4  {'bytes_in_flight': 2354, 'packets_in_flight': 3}  1.710329e+12   

                    timestamp  
0  2024-03-13 11:30:24.379761  
1  2024-03-13 11:30:24.379765  
2  2024-03-13 11:30:24.405572  
3  2024-03-13 11:30:24.405845  
4  2024-03-13 11:30:24.407439  
        time                   name  \
0  28.160371  transport:packet_sent   
1  29.748164  transport:packet_sent   
2  31.330857  tra

In [855]:
ori_recover_c_len = len(metrics_sent_rows)
for i in range(pk_sent_cnt):
    if(i >= len(metrics_sent_rows)):
        data = metrics_sent_rows.iloc[i-1]['data']
        new_row_data = {'time': [pk_sent_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_sent_rows = pd.concat([metrics_sent_rows, new_row], ignore_index=True)
        continue
    time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']
    # print(i, time_diff)
    # time_diff >= 1: not the matching metrics_update
    while time_diff >= 1:
        data = metrics_sent_rows.iloc[i-1]['data']
        new_row_data = {'time': [pk_sent_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        # print(new_row)
        metrics_sent_rows = insert(metrics_sent_rows, i, new_row)
        time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']
    # time_diff < 0: missing metrics_update
    while time_diff < 0:
        # print(i, time_diff_list)
        metrics_sent_rows.drop(index=metrics_sent_rows.index[i], inplace=True)
        time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']

    

# if len(metrics_sent_rows) < pk_sent_cnt:
#     d = pk_sent_cnt - len(metrics_sent_rows)
# data = metrics_sent_rows.iloc[len(metrics_sent_rows)-1]['data']

# for i in range(d):
#     last_row_data = {'time': [pk_sent_rows.iloc[len(metrics_sent_rows)-1]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
#     new_row_df = pd.DataFrame(last_row_data)
#     metrics_sent_rows = pd.concat([metrics_sent_rows, new_row], ignore_index=True)

print(ori_recover_c_len, len(metrics_sent_rows))


281607 281618


In [856]:
metrics_sent_rows = metrics_sent_rows.reset_index(drop=True)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)
print(len(metrics_sent_rows), len(pk_sent_rows))

# check whether there's still mismatch exist.
time_diff_list = metrics_sent_rows['time'] - pk_sent_rows['time']
mismatch_indices = time_diff_list[(time_diff_list >= 1) | (time_diff_list < 0)].index
if len(mismatch_indices) == 0:
    print("All Matched!")
else:
    print(mismatch_indices)


281618 281618
All Matched!


In [857]:
# extract bytes_in_flight & packets_in_flight
metrics_sent_rows['bytes_in_flight'] = None
metrics_sent_rows['packets_in_flight'] = None

# Use ast.literal_eval to safely evaluate the string and extract 'bytes_in_flight' and 'packets_in_flight'
metrics_sent_rows[['bytes_in_flight', 'packets_in_flight']] = metrics_sent_rows['data'].apply(
    lambda x: pd.Series(ast.literal_eval(x)) if isinstance(x, str) else pd.Series([None, None]))

metrics_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight
0,28.165609,recovery:metrics_updated,"{'bytes_in_flight': 1331, 'packets_in_flight': 2}",1.710329e+12,2024-03-13 11:30:24.405845,1331,2
1,29.759548,recovery:metrics_updated,"{'bytes_in_flight': 2354, 'packets_in_flight': 3}",1.710329e+12,2024-03-13 11:30:24.407439,2354,3
2,31.337423,recovery:metrics_updated,"{'bytes_in_flight': 3377, 'packets_in_flight': 4}",1.710329e+12,2024-03-13 11:30:24.409017,3377,4
3,32.933317,recovery:metrics_updated,"{'bytes_in_flight': 4400, 'packets_in_flight': 5}",1.710329e+12,2024-03-13 11:30:24.410612,4400,5
4,34.534100,recovery:metrics_updated,"{'bytes_in_flight': 5423, 'packets_in_flight': 6}",1.710329e+12,2024-03-13 11:30:24.412213,5423,6


In [858]:
# Add bytes_in_flight & packets_in_flight to pk_sent_rows
pk_sent_rows['bytes_in_flight'] = metrics_sent_rows['bytes_in_flight']
pk_sent_rows['packets_in_flight'] = metrics_sent_rows['packets_in_flight']

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight
0,28.160371,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.405840,1331,2
1,29.748164,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.407427,2354,3
2,31.330857,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.409010,3377,4
3,32.926612,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.410606,4400,5
4,34.527465,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412207,5423,6


Concat `transport:packet_received` & `recovery:metrics_updated`.

In [859]:
metrics_ack_csv_file_path = path + "middle/" + f"ack_metrics_{time}_{port}.csv" 
metrics_ack_rows.to_csv(metrics_ack_csv_file_path, index=False)
rcv_ack_csv_file_path = path + "middle/" + f"rcv_ack_{time}_{port}.csv"
rcv_ack_rows.to_csv(rcv_ack_csv_file_path, index=False)

In [860]:
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)
initial_ack_metrics = metrics_ack_rows.iloc[[0]]
metrics_ack_rows.drop(index=metrics_ack_rows.index[0], inplace=True)
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)

In [861]:
metrics_ack_rows[:3]

,time,name,data,epoch_time,timestamp
0,27.540731,recovery:metrics_updated,"{'min_rtt': 25.43443, 'smoothed_rtt': 25.43443...",1.710329e+12,2024-03-13 11:30:24.405220
1,27.802779,recovery:metrics_updated,"{'smoothed_rtt': 25.436, 'latest_rtt': 25.4551...",1.710329e+12,2024-03-13 11:30:24.405482
2,64.031529,recovery:metrics_updated,"{'smoothed_rtt': 26.308, 'latest_rtt': 32.4131...",1.710329e+12,2024-03-13 11:30:24.441711


In [862]:
rcv_ack_rows[:3]

,time,name,data,epoch_time,timestamp
0,27.810252,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.405489
1,64.039980,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.441719
2,64.058698,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.441738


In [863]:
print(len(metrics_ack_rows), len(rcv_ack_rows))
for i in range(rcv_ack_cnt):
    if(i >= len(metrics_ack_rows)):
        data = metrics_ack_rows.iloc[i-1]['data']
        new_row_data = {'time': [rcv_ack_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_ack_rows = pd.concat([metrics_ack_rows, new_row], ignore_index=True)
        continue
    time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
    # time_diff >= 1: not the matching metrics_update
    while time_diff > 0:
        # print("> 0:", i, time_diff)
        if i == 0:
            data = initial_ack_metrics.iloc[0]['data']
        else:
            data = metrics_ack_rows.iloc[i-1]['data']
        new_row_data = {'time': [rcv_ack_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_ack_rows = insert(metrics_ack_rows, i, new_row)
        time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
    # time_diff < 0: missing metrics_update
    while time_diff <= -1:
        # print("<= -1:", i, time_diff)
        metrics_ack_rows.drop(index=metrics_ack_rows.index[i], inplace=True)
        time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
print(len(metrics_ack_rows), len(rcv_ack_rows))

65206 65337
65337 65337


In [864]:
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

# check whether there's still mismatch exist.
time_diff_list = metrics_ack_rows['time'] - rcv_ack_rows['time']
mismatch_indices = time_diff_list[(time_diff_list <= -1) | (time_diff_list > 0)].index
if len(mismatch_indices) == 0:
    print("All Matched!")
else:
    print(mismatch_indices)

All Matched!


In [865]:
ack_json_list = []
## Add the initial_ack_metrics for temporary
print(initial_ack_metrics)
metrics_ack_rows = pd.concat([initial_ack_metrics, metrics_ack_rows], axis=0).reset_index(drop=True)
for i in range(len(metrics_ack_rows)):
    s = metrics_ack_rows.iloc[i]['data'].replace("\'", "\"")
    json_object = json.loads(s)
    ack_json_list.append(json_object)

metrics_ack_df = pd.DataFrame(ack_json_list)
# Fill missing values in each row with the previous row's values
metrics_ack_df = metrics_ack_df.ffill(axis=0)

## drop initial_ack_metrics
metrics_ack_rows.drop(index=metrics_ack_rows.index[0], inplace=True)
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
metrics_ack_df.drop(index=metrics_ack_df.index[0], inplace=True)
metrics_ack_df = metrics_ack_df.reset_index(drop=True)
metrics_ack_df[:5]

       time                      name  \
0  2.073056  recovery:metrics_updated   

                                                data    epoch_time  \
0  {'min_rtt': 0, 'smoothed_rtt': 0, 'latest_rtt'...  1.710329e+12   

                    timestamp  
0  2024-03-13 11:30:24.379752  


,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
0,25.434430,25.43443,25.434430,12.717215,40064.0,823,2.0
1,25.434430,26.30800,32.413173,8.900000,45072.0,20472,20.0
2,25.434430,26.26600,25.974994,6.758000,45072.0,16380,16.0
3,21.494842,25.66900,21.494842,6.261000,45072.0,13311,13.0
4,21.494842,25.62000,25.278423,4.793000,45072.0,15369,15.0


In [866]:
metrics_ack_rows = pd.concat([metrics_ack_rows, metrics_ack_df], axis=1).reset_index(drop=True)
# since we have parse out all the information in data, we can drop the data cl=olumn
metrics_ack_rows = metrics_ack_rows.drop(columns=['data'])
metrics_ack_rows[:5]

,time,name,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
0,27.540731,recovery:metrics_updated,1.710329e+12,2024-03-13 11:30:24.405220,25.434430,25.43443,25.434430,12.717215,40064.0,823,2.0
1,64.031529,recovery:metrics_updated,1.710329e+12,2024-03-13 11:30:24.441711,25.434430,26.30800,32.413173,8.900000,45072.0,20472,20.0
2,64.052552,recovery:metrics_updated,1.710329e+12,2024-03-13 11:30:24.441732,25.434430,26.26600,25.974994,6.758000,45072.0,16380,16.0
3,64.081117,recovery:metrics_updated,1.710329e+12,2024-03-13 11:30:24.441760,21.494842,25.66900,21.494842,6.261000,45072.0,13311,13.0
4,74.331646,recovery:metrics_updated,1.710329e+12,2024-03-13 11:30:24.452011,21.494842,25.62000,25.278423,4.793000,45072.0,15369,15.0


In [867]:
# Check whehter the length is equal before concating metrics into rcv_ack_rows
print(len(rcv_ack_rows), len(metrics_ack_df))

65337 65337


In [868]:
rcv_ack_rows = pd.concat([rcv_ack_rows, metrics_ack_df], axis=1)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

print(len(rcv_ack_rows), len(metrics_ack_df))
rcv_ack_rows[-5:]

65337 65337


,time,name,data,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
65332,450023.787287,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.401467,10.651562,19.705,16.101058,2.992,58059.0,10250,10.0
65333,450029.224361,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.406904,10.651562,19.335,16.752825,2.982,58059.0,11275,11.0
65334,450039.422578,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.417102,10.651562,19.684,22.132108,2.935,58059.0,8200,8.0
65335,450043.776750,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.421456,10.651562,20.132,23.268009,3.097,58059.0,6150,6.0
65336,450053.787161,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710330e+12,2024-03-13 11:37:54.431466,10.651562,20.764,25.194722,3.588,58059.0,0,6.0


Mapping the ACK ranges

In [869]:
acked_ranges_series = rcv_ack_rows['data']
acked_ranges_list = []
for i in range(len(acked_ranges_series)):
    s = acked_ranges_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    # Extract 'acked_ranges' from all frames
    acked_ranges = [range_entry for frame in data_dict['frames'] if 'acked_ranges' in frame for range_entry in frame['acked_ranges']]
    acked_ranges_list.append(acked_ranges)

acked_ranges_df = pd.DataFrame({"acked_ranges": acked_ranges_list})
acked_ranges_df[:5]

,acked_ranges
0,[[0]]
1,"[[0, 4]]"
2,"[[0, 8]]"
3,"[[0, 11]]"
4,"[[0, 15]]"


In [870]:
rcv_ack_rows = pd.concat([rcv_ack_rows, acked_ranges_df], axis=1)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

rcv_ack_rows[:5]

,time,name,data,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight,acked_ranges
0,27.810252,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.405489,25.434430,25.43443,25.434430,12.717215,40064.0,823,2.0,[[0]]
1,64.039980,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.441719,25.434430,26.30800,32.413173,8.900000,45072.0,20472,20.0,"[[0, 4]]"
2,64.058698,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.441738,25.434430,26.26600,25.974994,6.758000,45072.0,16380,16.0,"[[0, 8]]"
3,64.087333,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.441767,21.494842,25.66900,21.494842,6.261000,45072.0,13311,13.0,"[[0, 11]]"
4,74.343449,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.452023,21.494842,25.62000,25.278423,4.793000,45072.0,15369,15.0,"[[0, 15]]"


In [871]:
# parse out the packet_number & offset & length
pk_sent_series =  pk_sent_rows['data']
pk_num_list = []
offset_list = []
length_list = []
for i in range(len(pk_sent_series)):
    s = pk_sent_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    packet_number = data_dict['header']['packet_number']
    # Initialize offset to None in case 'frame_type': 'stream' is not found
    offset = None
    # Iterate through frames to find 'offset' for 'frame_type': 'stream'
    for frame in data_dict.get('frames', []):
        if frame.get('frame_type') == 'stream':
            offset = frame.get('offset')
            length = frame.get('length')
            break  # Stop iterating once 'offset' is found
    
    pk_num_list.append(packet_number)
    offset_list.append(offset)
    length_list.append(length)

pk_num_df = pd.DataFrame({"packet_number": pk_num_list, "offset": offset_list, "length": length_list})
pk_num_df[:5]

,packet_number,offset,length
0,2,0,1000
1,3,1000,1000
2,4,2000,1000
3,5,3000,1000
4,6,4000,1000


In [872]:
pk_sent_rows = pd.concat([pk_sent_rows, pk_num_df], axis=1)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length
0,28.160371,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.405840,1331,2,2,0,1000
1,29.748164,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.407427,2354,3,3,1000,1000
2,31.330857,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.409010,3377,4,4,2000,1000
3,32.926612,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.410606,4400,5,5,3000,1000
4,34.527465,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412207,5423,6,6,4000,1000


In [873]:
pk_sent_rows['smoothed_rtt'] = np.nan
pk_sent_rows['latest_rtt'] = np.nan
pk_sent_rows['rtt_variance'] = np.nan
pk_sent_rows['congestion_window'] = np.nan

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window
0,28.160371,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.405840,1331,2,2,0,1000,NaN,NaN,NaN,NaN
1,29.748164,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.407427,2354,3,3,1000,1000,NaN,NaN,NaN,NaN
2,31.330857,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.409010,3377,4,4,2000,1000,NaN,NaN,NaN,NaN
3,32.926612,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.410606,4400,5,5,3000,1000,NaN,NaN,NaN,NaN
4,34.527465,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412207,5423,6,6,4000,1000,NaN,NaN,NaN,NaN


In [874]:
def update_pk_sent_rows(row):
    acked_ranges = row['acked_ranges']
    smoothed_rtt = row['smoothed_rtt']
    latest_rtt = row['latest_rtt']
    rtt_variance = row['rtt_variance']
    congestion_window = row['congestion_window']

    for ack_range in acked_ranges:
        start_packet, end_packet = ack_range[0], ack_range[-1]
        existing_packets = set(pk_sent_rows['packet_number'])
        packet_numbers_to_update = set(range(start_packet, end_packet + 1)).intersection(existing_packets)

        mask = pk_sent_rows['packet_number'].isin(packet_numbers_to_update)
        pk_sent_rows.loc[mask, 'smoothed_rtt'] = pk_sent_rows.loc[mask, 'smoothed_rtt'].fillna(smoothed_rtt)
        pk_sent_rows.loc[mask, 'latest_rtt'] = pk_sent_rows.loc[mask, 'latest_rtt'].fillna(latest_rtt)
        pk_sent_rows.loc[mask, 'congestion_window'] = pk_sent_rows.loc[mask, 'congestion_window'].fillna(congestion_window)
        pk_sent_rows.loc[mask, 'rtt_variance'] = pk_sent_rows.loc[mask, 'rtt_variance'].fillna(rtt_variance)

# Apply the custom update function to each row in rcv_ack_rows
rcv_ack_rows.apply(update_pk_sent_rows, axis=1)

# Display the updated pk_sent_rows
pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window
0,28.160371,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.405840,1331,2,2,0,1000,26.308,32.413173,8.900,45072.0
1,29.748164,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.407427,2354,3,3,1000,1000,26.308,32.413173,8.900,45072.0
2,31.330857,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.409010,3377,4,4,2000,1000,26.308,32.413173,8.900,45072.0
3,32.926612,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.410606,4400,5,5,3000,1000,26.266,25.974994,6.758,45072.0
4,34.527465,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412207,5423,6,6,4000,1000,26.266,25.974994,6.758,45072.0


Identify lost packets

In [875]:
# Use ast.literal_eval to safely evaluate the string and extract 'packet_number'
lost_rows['packet_number'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['header']['packet_number'] if isinstance(x, str) else None)
lost_rows['trigger'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['trigger'] if isinstance(x, str) else None)
lost_rows[:5]

/var/folders/ll/z00qrhv13397cv4_52zrrb200000gn/T/ipykernel_28184/2583690328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lost_rows['packet_number'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['header']['packet_number'] if isinstance(x, str) else None)
/var/folders/ll/z00qrhv13397cv4_52zrrb200000gn/T/ipykernel_28184/2583690328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lost_rows['trigger'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['trigger'] if isinstance(x, str) els

,time,name,data,epoch_time,timestamp,packet_number,trigger
39417,17719.336566,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:42.097016,10894,reordering_threshold
93254,41123.991355,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:31:05.501671,25586,reordering_threshold
117888,51915.142183,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:31:16.292821,32273,reordering_threshold
117890,51915.153568,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:31:16.292833,32274,reordering_threshold
117895,51915.176965,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:31:16.292856,32275,reordering_threshold


In [876]:
lost_pk_csv_file_path = path + "middle/" + f"lost_pk_{time}_{port}.csv"
lost_rows.to_csv(lost_pk_csv_file_path, index=False)

In [877]:
## set to True if the packet is lost
pk_sent_rows['packet_lost'] = False

# Iterate through rows and set 'packet_lost' to True where 'packet_number' values match
for _, lost_row in lost_rows.iterrows():
    packet_number = lost_row['packet_number']
    
    # Check if 'packet_number' exists in pk_sent_rows
    if packet_number in pk_sent_rows['packet_number'].values:
        pk_sent_rows.loc[pk_sent_rows['packet_number'] == packet_number, 'packet_lost'] = True

pk_sent_rows[19340:19345]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,packet_lost
19340,30972.369813,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:55.350049,14350,14,19364,19339000,1000,19.516,19.683406,2.049,90995.0,False
19341,30973.969269,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:55.351649,15375,15,19365,19340000,1000,19.516,19.683406,2.049,90995.0,False
19342,30975.553500,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:55.353233,16400,16,19366,19341000,1000,19.386,18.477206,1.796,90995.0,False
19343,30977.196747,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:55.354876,17425,17,19367,19342000,1000,19.386,18.477206,1.796,90995.0,False
19344,30978.798577,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:55.356478,18450,18,19368,19343000,1000,19.386,18.477206,1.796,90995.0,False


In [878]:
cols = ['time', 'epoch_time', 'timestamp', 'name', 'packet_number', 'offset', 'length', 'bytes_in_flight', 'packets_in_flight', 'smoothed_rtt', 'latest_rtt', 'rtt_variance', 'congestion_window', 'packet_lost', 'data']
processed_df = pk_sent_rows[cols]
processed_df[:5]

,time,epoch_time,timestamp,name,packet_number,offset,length,bytes_in_flight,packets_in_flight,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,packet_lost,data
0,28.160371,1.710329e+12,2024-03-13 11:30:24.405840,transport:packet_sent,2,0,1000,1331,2,26.308,32.413173,8.900,45072.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."
1,29.748164,1.710329e+12,2024-03-13 11:30:24.407427,transport:packet_sent,3,1000,1000,2354,3,26.308,32.413173,8.900,45072.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."
2,31.330857,1.710329e+12,2024-03-13 11:30:24.409010,transport:packet_sent,4,2000,1000,3377,4,26.308,32.413173,8.900,45072.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."
3,32.926612,1.710329e+12,2024-03-13 11:30:24.410606,transport:packet_sent,5,3000,1000,4400,5,26.266,25.974994,6.758,45072.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."
4,34.527465,1.710329e+12,2024-03-13 11:30:24.412207,transport:packet_sent,6,4000,1000,5423,6,26.266,25.974994,6.758,45072.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."


In [879]:
csv_file_path = path + "data/" + f"processed_sent_{time}_{port}.csv"
processed_df.to_csv(csv_file_path, sep='@', index=False)

## Receiver side data

In [880]:
pk_rcv_df = pk_rcv_rows.reset_index(drop=True)
pk_rcv_df[:5]

,time,name,data,epoch_time,timestamp
0,56.321354,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412239
1,56.355989,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412273
2,56.374896,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412292
3,60.963594,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.416881
4,60.996302,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.416914


In [881]:
pk_rcv_series =  pk_rcv_df['data']
pk_rcv_num_list = []
offset_rcv_list = []
length_rcv_list = []
for i in range(len(pk_rcv_series)):
    s = pk_rcv_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    packet_number = data_dict['header']['packet_number']
    # Initialize offset to None in case 'frame_type': 'stream' is not found
    offset = None
    # Iterate through frames to find 'offset' for 'frame_type': 'stream'
    for frame in data_dict.get('frames', []):
        if frame.get('frame_type') == 'stream':
            offset = frame.get('offset')
            length = frame.get('length')
            break  # Stop iterating once 'offset' is found
    
    pk_rcv_num_list.append(packet_number)
    offset_rcv_list.append(offset)
    length_rcv_list.append(length)

pk_rcv_df['packet_number'] = pk_rcv_num_list
pk_rcv_df['offset'] = offset_rcv_list
pk_rcv_df['length'] = length_rcv_list

pk_rcv_df[:5]

,time,name,data,epoch_time,timestamp,packet_number,offset,length
0,56.321354,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412239,2,0,1000
1,56.355989,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412273,3,1000,1000
2,56.374896,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.412292,4,2000,1000
3,60.963594,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.416881,5,3000,1000
4,60.996302,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.710329e+12,2024-03-13 11:30:24.416914,6,4000,1000


In [882]:
cols = ['time', 'epoch_time', 'timestamp', 'name', 'packet_number', 'offset', 'length', 'data']
processed_rcv_df = pk_rcv_df[cols]
processed_rcv_df[:5]


,time,epoch_time,timestamp,name,packet_number,offset,length,data
0,56.321354,1.710329e+12,2024-03-13 11:30:24.412239,transport:packet_received,2,0,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."
1,56.355989,1.710329e+12,2024-03-13 11:30:24.412273,transport:packet_received,3,1000,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."
2,56.374896,1.710329e+12,2024-03-13 11:30:24.412292,transport:packet_received,4,2000,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."
3,60.963594,1.710329e+12,2024-03-13 11:30:24.416881,transport:packet_received,5,3000,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."
4,60.996302,1.710329e+12,2024-03-13 11:30:24.416914,transport:packet_received,6,4000,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."


In [883]:
csv_file_path = path + "data/" + f"processed_rcv_{time}_{port}.csv"
processed_rcv_df.to_csv(csv_file_path, sep='@')

## Real Packet Loss Data

In [7]:
### USER SETTINGS ###
database="/Volumes/mollyT7/MOXA/"
# database = "/Volumes/MOLLY256/MOXA/"
# database = "/Users/molly/Desktop"
dates = [
    "2024-03-13",
]
devices = sorted([
    "sm00",
    "sm01",
])
exps = {
    # "QUIC-450sec": (6, ["#{:02d}".format(i + 1) for i in range(6)]),
    # "QUIC-300sec": (6, ["#{:02d}".format(i + 1) for i in range(6)]),
    "QUIC-1M": (2, ["#{:02d}".format(i + 1) for i in range(2)]),
    "QUIC-5M": (2, ["#{:02d}".format(i + 1) for i in range(2)]),
    "QUIC-10M": (2, ["#{:02d}".format(i + 1) for i in range(2)]),
}

device_to_port = {"sm00": [5200, 5201], 
                  "sm01": [5202, 5203],
                  "sm02": [5204, 5205]}

In [8]:
def find_ul_file(database, date, exp, device):
    ul_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'data')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "processed_rcv" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[0]) in numbers:
                        ul_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return ul_files

def find_dl_file(database, date, exp, device):
    dl_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'data')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "processed_rcv" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[1]) in numbers:
                        dl_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return dl_files

def find_ul_loss_file(database, date, exp, device):
    ul_loss_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'middle')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "lost_pk" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[0]) in numbers:
                        ul_loss_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return ul_loss_files

def find_dl_loss_file(database, date, exp, device):
    dl_loss_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'middle')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "lost_pk" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[1]) in numbers:
                        dl_loss_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return dl_loss_files


In [9]:
def get_loss_data(lost_df, received_df):
    # Check if each row in ul_lost_df['packet_number'] is present in ul_received_df['packet_number']
    lost_in_received = lost_df['packet_number'].isin(received_df['packet_number'])

    # Get the rows in ul_lost_df where the packet number is present in ul_received_df
    exec_lat_df = lost_df[lost_in_received]

    exec_reorder_df = exec_lat_df[exec_lat_df['trigger'] == 'reordering_threshold']
    exec_time_df = exec_lat_df[exec_lat_df['trigger'] == 'time_threshold']

    # Get the rows in ul_lost_df where the packet number is not present in ul_received_df
    real_lost_df = lost_df[~lost_in_received]

    # Filter ul_lost_df for rows where 'trigger' is 'reordering threshold'
    lost_reorder_df = real_lost_df[real_lost_df['trigger'] == 'reordering_threshold']
    lost_time_df = real_lost_df[real_lost_df['trigger'] == 'time_threshold']

    return exec_lat_df, exec_reorder_df, exec_time_df, real_lost_df, lost_reorder_df, lost_time_df

def calculate_statistics(lost_reorder_df, lost_time_df, real_lost_df, exec_reorder_df, exec_time_df, exec_lat_df, lost_df, df):
    statistics_data = {
        'original_pkl': len(lost_df),
        'reordering_threshold': len(lost_reorder_df),
        'time_threshold': len(lost_time_df),
        'real_pkl': len(real_lost_df),
        'exec_reordering': len(exec_reorder_df),
        'exec_time': len(exec_time_df),
        'exec_lat': len(exec_lat_df),
        'reordering_pkl_rate(%)': 0 if len(real_lost_df) == 0 else len(lost_reorder_df)*100 / len(real_lost_df),
        'time_pkl_rate(%)': 0 if len(real_lost_df) == 0 else len(lost_time_df)*100 / len(real_lost_df),
        'real_pkl_rate(%)': 0 if len(lost_df) == 0 else len(real_lost_df)*100 / len(lost_df),
        'original_packet_loss_rate(%)': len(lost_df)*100 / (df.iloc[-1]['offset'] / 250),
        'adjusted_packet_loss_rate(%)': len(real_lost_df)*100 / (df.iloc[-1]['offset'] / 250)
    }

    # Convert the dictionary to a dataframe
    statistics_df = pd.DataFrame(statistics_data, index=[0])
    return statistics_df

In [10]:
# exec_lat_df, exec_reorder_df, exec_time_df, real_lost_df, lost_reorder_df, lost_time_df = get_loss_data(lost_rows, processed_rcv_df)

In [11]:
all_ul_files = []
all_ul_pkl_files = []
for date in dates:
    for exp in exps:
        for device in devices:
            ul_pk_files = find_ul_file(database, date, exp, device)
            ul_pk_loss_files = find_ul_loss_file(database, date, exp, device)
            all_ul_files.extend(ul_pk_files)
            all_ul_pkl_files.extend(ul_pk_loss_files)

for i in range(len(all_ul_files)):
    ul_rcv_df = pd.read_csv(all_ul_files[i], sep='@')
    ul_loss_df = pd.read_csv(all_ul_pkl_files[i])
    ul_exec_lat_df, ul_exec_reorder_df, ul_exec_time_df, ul_real_lost_df, ul_lost_reorder_df, ul_lost_time_df = get_loss_data(ul_loss_df, ul_rcv_df)
    ul_statistics = calculate_statistics(ul_lost_reorder_df, ul_lost_time_df, ul_real_lost_df, ul_exec_reorder_df, ul_exec_time_df, ul_exec_lat_df, ul_loss_df, ul_rcv_df)

    directory = os.path.dirname(all_ul_files[i])
    ul_loss_df['lost'] = False
    ul_loss_df['excl'] = False
    # Set 'lost' column to True for rows in ul_real_lost_df
    ul_loss_df.loc[ul_loss_df['packet_number'].isin(ul_real_lost_df['packet_number']), 'lost'] = True
    # Set 'excl' column to True for rows in ul_exec_lat_df
    ul_loss_df.loc[ul_loss_df['packet_number'].isin(ul_exec_lat_df['packet_number']), 'excl'] = True
    ul_loss_df.to_csv(f"{directory}/ul_real_lost_pk.csv", index=False)
    # modify to the statistics directory
    parts = directory.split("/")
    parts[-1] = "statistics"
    statistics_directory = "/".join(parts)
    ul_statistics.to_csv(f"{statistics_directory}/ul_statistics.csv", index=False)

In [12]:
all_dl_files = []
all_dl_pkl_files = []
for date in dates:
    for exp in exps:
        for device in devices:
            dl_pk_files = find_dl_file(database, date, exp, device)
            dl_pk_loss_files = find_dl_loss_file(database, date, exp, device)
            all_dl_files.extend(dl_pk_files)
            all_dl_pkl_files.extend(dl_pk_loss_files)

for i in range(len(all_dl_files)):
    dl_rcv_df = pd.read_csv(all_dl_files[i], sep='@')
    dl_loss_df = pd.read_csv(all_dl_pkl_files[i])
    dl_exec_lat_df, dl_exec_reorder_df, dl_exec_time_df, dl_real_lost_df, dl_lost_reorder_df, dl_lost_time_df = get_loss_data(dl_loss_df, dl_rcv_df)
    dl_statistics = calculate_statistics(dl_lost_reorder_df, dl_lost_time_df, dl_real_lost_df, dl_exec_reorder_df, dl_exec_time_df, dl_exec_lat_df, dl_loss_df, dl_rcv_df)
    
    directory = os.path.dirname(all_dl_files[i])
    dl_loss_df['lost'] = False
    dl_loss_df['excl'] = False
    # Set 'lost' column to True for rows in dl_real_lost_df
    dl_loss_df.loc[dl_loss_df['packet_number'].isin(dl_real_lost_df['packet_number']), 'lost'] = True
    # Set 'excl' column to True for rows in ul_exec_lat_df
    dl_loss_df.loc[dl_loss_df['packet_number'].isin(dl_exec_lat_df['packet_number']), 'excl'] = True
    dl_loss_df.to_csv(f"{directory}/dl_real_lost_pk.csv", index=False)
    # modify to the statistics directory
    parts = directory.split("/")
    parts[-1] = "statistics"
    statistics_directory = "/".join(parts)
    dl_statistics.to_csv(f"{statistics_directory}/dl_statistics.csv", index=False)